In [ ]:
# Learning with inspiration from the below
# https://www.kaggle.com/mehmetzahidgen/the-three-hair-types-cnn-with-pytorch
# and https://www.youtube.com/watch?v=jZVVFUhsbgY&t=977s

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
image_dir = Path('../input/different-hair-types')

In [ ]:
image_dir

# Create File DataFrame

In [ ]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name = 'Filepath', dtype = np.uint8 ).astype(str)
labels = pd.Series(labels, name= 'Label', dtype = np.uint8)

image_df = pd.concat([filepaths, labels], axis = 1)

In [ ]:
image_df

In [ ]:
train_df, test_df = train_test_split(image_df, train_size = 0.7, shuffle = True, random_state = 1)

In [ ]:
train_df

# Load Image Data

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
rescale = 1. / 255,
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
rescale = 1. / 255
)



In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'training'
    
)

val_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
    
)

test_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = False,
 
    
)

# Training

In [ ]:
inputs = tf.keras.Input(shape=(120,120, 7))


x = tf.keras.layers.Conv2D(filters = 14, kernel_size = (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 28, kernel_size=(3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)

outputs= tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
   metrics = ['accuracy']
)

history = model.fit(
    train_images,
    validation_data = val_images,
    epochs = 100,
   callbacks= 
    [
        tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience = 5,
        restore_best_weights = True
        ),
       
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        patience = 3
            ) 
              
       ]
    )


* 

In [ ]:
#train_images.class_indices

# Results

In [ ]:
results = model.evaluate(test_images, verbose = 0)

print("Test Loss: {:.5f}".format(results[0]))

print("Test Accuracy: {:.2f}".format(results[1] * 100))



In [ ]:
predictions = model.predict(test_images) >= 0.5
